This part demonstrates the process of checking whether the song lyrics contain more male or female words.
This presentation will not follow the code flow of the component.

First step is to ask the user for a few input parameters, such as: the name of the item he's searching for, what kind of item it is(track/album/artist) and his spotify token which can be generated on the spotify developers website. The user's input then gets added to the search() function as its parameters and returns all the songs which have corresponding keywords in their name. 

In [12]:
import requests
import json
from bs4 import BeautifulSoup
from collections import Counter
GENIUS_TOKEN = "ccHDzjsPAnwWcOAo2fS5Kb5cH9BFLqsryi5r0a6p3flgb_B3g8qPguO68A3NZTZJ"

def search(item_name, item_type, token):
    url = f"https://api.spotify.com/v1/search"
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    params = {
        "q": item_name,
        "type": item_type
    }
    r = requests.get(url, headers=headers,
                     params=params)
    data = r.json()
    tracks = data["tracks"]["items"]
    return [(track["name"], track["artists"][0]["name"]) for track in tracks]

def read_from_console():
    input_q = input("provide a name of an item you are searching for: \n> ")
    input_type = input("provide a category album/artist/playlist/track: \n> ")
    token = input("Spotify authentification token: \n> ")
    tracks_info = search(input_q, input_type, token)
    
    for song, artist in tracks_info:
        print(f"{song}  -  {artist}")

read_from_console()


provide a name of an item you are searching for: 
> Over Now
provide a category album/artist/playlist/track: 
> track
Spotify authentification token: 
> BQCQRhjCk4sA7KyhihoZu3yHMEYepquAhuY18bltAuGLU-xKwAG17OManzkgVtX8aGAB4IWoPYnUxjy0SYyffsW1yQIHKeSBy8pgjbQeeo5ml-jKDOlTn130Nq_xDJOIos8amTRkg9tZLWvBmOHok2dYzO_f-YFy7lXQwZscnq9aeppprHdGK067i0SnMAzzmf6Kitl4s--1DtKCMItDoLG0jvbPKkh_yX9fBeNNFvR7xQ1PvD57jEDwyOMRw0ITLQRUqk1uZM6i6dOB5A
Over Now  -  Post Malone
Now It's Over. Now I'm Awake.  -  LUCHS
Over Now  -  Primacy Funk
Over Now  -  Andreas Wistrand
The Storm Is Over Now  -  R. Kelly
Now Or Never  -  The Roots
All Over Now  -  Eric Hutchinson
What About Now  -  Daughtry
It's All over Now, Baby Blue  -  Bob Dylan
Starting Over, Starting Now - From "The Lodge"  -  Cast of The Lodge
Over Now  -  Alice In Chains
Over Now - Live at the Majestic Theatre, Brooklyn, NY - April 1996  -  Alice In Chains
Both Sides Now  -  Susan Boyle
Big Girl Now  -  Anna David
All Over Now  -  Washed Out
Starting Over

After that, we'll ask a user to choose an artist he wants and we'll put his input in the request_song_info() as a parameter along with the track name which he provided before hand and we'll jsonify it so the response looks presentable.
Due to the object we got from the request, we will have to iterate over the "hits" key in that object and look for an exact match using the artist_name variable. Since there is no endpoint on the Genius API for lyrics, we have to extract the url of the song and use scrap_song_url() function to crawl the lyrics for us.

In [13]:
import requests
import json
from bs4 import BeautifulSoup
from collections import Counter
GENIUS_TOKEN = "ccHDzjsPAnwWcOAo2fS5Kb5cH9BFLqsryi5r0a6p3flgb_B3g8qPguO68A3NZTZJ"


def request_song_info(song_title, artist_name):
    url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' +
               GENIUS_TOKEN}
    search_url = url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response


def scrap_song_url(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    return lyrics


def read_from_console():   
    input_artist = input("Choose an artist:")
    response = request_song_info("Over Now", input_artist)
    json = response.json()
    remote_song_info = None

    for hit in json['response']['hits']:
        if input_artist.lower() in hit['result']['primary_artist']['name'].lower():
            remote_song_info = hit
            break

    # Extract lyrics from URL if the song was found
    if remote_song_info:
        song_url = remote_song_info['result']['url']
        lyrics = scrap_song_url(song_url)
        print(lyrics)
read_from_console()

Choose an artist:Post Malone


[Intro]
Ahahah, yeah
(Yeah, yeah, yeah)

[Verse 1]
When you leave me, made me jump out of my room
Didn't faze you, would've given you the moon
Even after everything that we've been through
I ain't ever met nobody like you
Why you always tryna tell me that it's cool?
Need it quiet 'cause you never tell the truth
Felt like ballin' was the only thing to do
Trade my women for some diamonds and a coupe

[Pre-Chorus]
When she's comin' for my heart, it feels like Armageddon
I'ma turn the tables, promise you will not forget it
Won't apologize, don't give a fuck if you're offended
No discussion, yeah we gotta go our separate ways
Won't take me down in flames
I don't even wanna know your name
Watch how everything is gonna change, yeah, ayy

[Chorus]
I'ma pull up in a Bentley
With a fuckin' hundred thousand in my pocket, yeah, yeah
I'ma pull in, swang my door open
Stuntin', knowin' everybody watchin', yeah, yeah
And help another bitch out the whip
And I don't give a

In the last step of the component, we have 2 preset lists of words which are more often used by men and women.
We also have 2 empty dictionaries which we will use later in process. We iterate over the lyrics of the song and split them by space, checking if any of the words in lyrics also appear in our preset lists, if they do we set the certain word to be the first one that appeared with setdefault() method and add it to the dictionary. After that, we compare the sum of each of the dictionary's values and based on the result, we return that the song is predominatly for men or women.

In [14]:
lyrics = '''
Ahahah, yeah
(Yeah, yeah, yeah)

[Verse 1]
When you leave me, made me jump out of my room
Didn't faze you, would've given you the moon
Even after everything that we've been through
I ain't ever met nobody like you
Why you always tryna tell me that it's cool?
Need it quiet 'cause you never tell the truth
Felt like ballin' was the only thing to do
Trade my women for some diamonds and a coupe

[Pre-Chorus]
When she's comin' for my heart, it feels like Armageddon
I'ma turn the tables, promise you will not forget it
Won't apologize, don't give a fuck if you're offended
No discussion, yeah we gotta go our separate ways
Won't take me down in flames
I don't even wanna know your name
Watch how everything is gonna change, yeah, ayy

[Chorus]
I'ma pull up in a Bentley
With a fuckin' hundred thousand in my pocket, yeah, yeah
I'ma pull in, swang my door open
Stuntin', knowin' everybody watchin', yeah, yeah
And help another bitch out the whip
And I don't give a fuck if you don't like it, yeah, yeah
Just to show you that it's over
It don't matter what you gotta say about it, yeah
I'ma put that bitch pussy in a motherfucking bodybag
So you know that I'm never ever coming back
You kept your heart on the counter in a Prada bag
I used to never ever want to see the light of day, yeah
I was an idiot
Begging on my knees on to the floor
Now, I don't even want you anymore, anymore
It's over now
It's over now, yeah, yeah, yeah, ayy

[Verse 2]
Feels like a knife in my throat, you treat this like it's a joke
I bet you've done this before, I ain't surprised, yeah
I wasn't tryna let go, but you'll know why when I'm a ghost
I hope you wind up alone, dead in my eyes, yeah
Ain't religious, only Christian when it's Dior
Only prayin' for the Amex and the gold
You ain't stop me now; you only was a detour
Tried to drive me crazy, you ran out of road

[Pre-Chorus]
When she's comin' for my heart, it feels like Armageddon
I'ma turn the tables, promise you will not forget it
Won't apologize, don't give a fuck if you're offended
No discussion, yeah we gotta go our separate ways
Won't take me down in flames
I don't even wanna know your name
Watch how everything is gonna change, yeah, ayy

[Chorus]
I'ma pull up in a Bentley
With a fuckin' hundred thousand in my pocket, yeah, yeah
I'ma pull in, swang my door open
Stuntin', knowin' everybody watchin', yeah, yeah
And help another bitch out the whip
And I don't give a fuck if you don't like it, yeah, yeah
Just to show you that it's over
It don't matter what you gotta say about it, yeah
I'ma put that bitch pussy in a motherfucking bodybag
So you know that I'm never ever coming back
You kept your heart on the counter in a Prada bag
I used to never ever want to see the light of day, yeah
I was an idiot
Begging on my knees on to the floor
Now I don't even want you anymore, anymore
It's over now
It's over now
'''

def read_from_console():

    male_word_list = ["uh", "ah", "yeah", "mean", "you", "wife", "noise", "man", "hey", "pretty", "the",
                       "a", "of", "shit", "sort", "cool", "i", "like", "what", "guy", "there", "bucks", "nigga"]
    female_word_list = ["mhm", "husband", "and", "my", "oh", "she", "we", "um",
                         "have", "kids", "he", "her", "children", "because", "so",
                         "yes", "daughter", "gosh", "goodness", "son", "home", "too", "wow", "uh-huh"]

    male_word_count = {}
    female_word_count = {}

    for word in lyrics.split():
        if word in male_word_list:
            male_word_count.setdefault(word, 0)
            male_word_count[word] += 1
        elif word in female_word_list:
            female_word_count.setdefault(word, 0)
            female_word_count[word] += 1
    print(male_word_count)
    print(female_word_count)
    if sum(male_word_count.values()) > sum(female_word_count.values()):
        print("Seems like this song favours male specific words")
    elif sum(male_word_count.values()) < sum(female_word_count.values()):
        print("Seems like this songs favour female specific words")
    else:
        print("Well well well, it's a draw")
        
read_from_console()

{'yeah': 15, 'you': 21, 'of': 4, 'the': 15, 'like': 8, 'a': 17, 'what': 2}
{'my': 12, 'and': 2, 'we': 2}
Seems like this song favours male specific words
